Creating sentence embeddings using transition matrices that takes into account, the ordering of the words in the sentence. 

In the following simple example, we will use a vocabulary of nine words and implement the forward propagation part of a neural network that will predict the next word in a sentence (i.e sequence of words). A sentence embedding is created by multiplying each word by a recurrance matrix, and summing up the resulting vectors. The recurrance matrix is initialized as an identity matrix of size equal to the hidden neurons (which is the number of columns in the wieght matrix). The recurrance matrix is optimized via gradient descent when training the neural net.  

In [1]:
import numpy as np

def softmax(x):
    temp = np.exp(x)
    s = np.sum(temp, axis = 1, keepdims = True)
    return  temp / s  

vocab_size = 9
hidden_neurons = 3

# hidden layer weights intialized to zero
W0 = np.zeros(shape=(vocab_size, hidden_neurons))

# output layer weights initialed to random values
W1 = np.random.randn(hidden_neurons, vocab_size) 

# words vectors for our vocabulary (i.e. rows of the hidden layer weights matrix corresponding to each word in the vocabulary)
word_vecs = {}
word_vecs['yankees'] = W0[0:1]
word_vecs['bears'] = self.W0[1:2]
word_vecs['braves'] = W0[2:3]
word_vecs['red'] = W0[3:4]
word_vecs['sox'] = W0[4:5]
word_vecs['lose'] = W0[5:6]
word_vecs['defeat'] = W0[6:7]
word_vecs['beat'] = W0[7:8]
word_vecs['tie'] = W0[8:9]

# recurrance matrix initialized to identity 
rec = np.eye(3)


**Forward Propagation**

Given an input sequence of three words, the word_vector for the first word in the sequence is passed on by the first layer to the second layer where it is multiplied by the transition matrix and added to the next word vector in the sequence. The result is then passed on to the next layer where we repeat this process of multiplying the layer input by the traisition matrix and adding to the next word_vector. After the last word_vector in the sequence is added, we pass the result on to the output layer where it gets multiplied to the output layer weights and operated on by the softmax function to obtain the final prediction. 

In [2]:
input_sequence = ['red', 'sox', 'defeat']
layer_0 = word_vecs[input_sequence[0]]
layer_1 = np.dot(layer_0, rec) + word_vecs[input_sequence[1]]
layer_2 = np.dot(layer_1, rec) + word_vecs[input_sequence[2]]
pred = softmax(np.dot(layer_2, W1))

**Backpropagation**

Now we will do the backpropagation and compute gradients of the word_vectors and transition matrix

In [3]:
# the target word is 'yankees', whch is the first word in our vocab 
y = np.zeros(shape=(1,vocab_size))
y[0,0] = 1

# error
E = (pred - y) * (pred - y)

# dE/dP
dP = 2 * (pred - y)
W1_grad = np.dot(layer_2.T, dP)

# dE_dL2
dL2 = np.dot(dP, W1.T)

# dE_d(defeat)
d_defeat = dL2 * 1
W_defeat_grad = d_defeat

# dE_dL1
dL1 = np.dot(dL2, rec.T) 
rec_grad_2 = np.dot(layer_1.T, dL2)

# dE_d(sox)
d_sox = dL1 * 1
W_sox_grad = d_sox

# dE_dL0
dL0 = np.dot(dL1, rec.T)
rec_grad_1 = np.dot(layer_0.T, dL1)
W_red_grad = dL0 * 1




**Optimization**

Now update the weights and recurrance matrix via gradient descent

In [4]:
alpha = 0.01
W1 -= alpha * W1_grad
word_vecs['defeat'] -= alpha * W_defeat_grad
word_vecs['sox'] -= alpha * W_sox_grad
word_vecs['red'] -= alpha * W_red_grad
rec -= alpha * (rec_grad_2 + rec_grad_1) 


**Training Loop**

In [5]:
niters = 5
alpha = 2

for iter in range(niters):
    
    # forward pass
    input_sequence = ['red', 'sox', 'defeat']
    layer_0 = word_vecs[input_sequence[0]]
    layer_1 = np.dot(layer_0, rec) + word_vecs[input_sequence[1]]
    layer_2 = np.dot(layer_1, rec) + word_vecs[input_sequence[2]]
    pred = softmax(np.dot(layer_2, W1))

    # backpropagation

    # the target word is 'yankees', whch is the first word in our vocab 
    y = np.zeros(shape=(1,vocab_size))
    y[0,0] = 1

    # error
    error = (pred - y) * (pred - y)

    # dE/dP
    dP = 2 * (pred - y)
    W1_grad = np.dot(layer_2.T, dP)

    # dE_dL2
    dL2 = np.dot(dP, W1.T)

    # dE_d(defeat)
    d_defeat = dL2 * 1
    W_defeat_grad = d_defeat

    # dE_dL1
    dL1 = np.dot(dL2, rec.T) 
    rec_grad_2 = np.dot(layer_1.T, dL2)

    # dE_d(sox)
    d_sox = dL1 * 1
    W_sox_grad = d_sox

    # dE_dL0
    dL0 = np.dot(dL1, rec.T)
    rec_grad_1 = np.dot(layer_0.T, dL1)
    W_red_grad = dL0 * 1

    # optimization
    alpha = 0.01
    W1 -= alpha * W1_grad
    word_vecs['defeat'] -= alpha * W_defeat_grad
    word_vecs['sox'] -= alpha * W_sox_grad
    word_vecs['red'] -= alpha * W_red_grad
    rec -= alpha * (rec_grad_2 + rec_grad_1) 

    print(f"Iteration# {iter}, Error: {np.sum(error)}, prediction: {pred[0,0]}")


Iteration# 0, Error: 0.8599394106202588, prediction: 0.12577466958563557
Iteration# 1, Error: 0.8294627500797493, prediction: 0.1415802306435736
Iteration# 2, Error: 0.7970721061211786, prediction: 0.1587558717425521
Iteration# 3, Error: 0.7624172597616761, prediction: 0.1775500385536929
Iteration# 4, Error: 0.7251869429397199, prediction: 0.1982290913123447


Training this network to answer simple questions with the Babi dataset.

In [6]:
import numpy as np


In [7]:
# read training data from file
f = open('tasksv11/en/qa1_single-supporting-fact_train.txt', 'r')
raw = f.readlines()
f.close()

In [8]:
# tokenize the first 1000 senteneces (remove numbers, newline characters and punctuations)
tokens= []
for i, sentence in enumerate(raw[0:1000]):
    tokenized_sent = sentence.lower().replace("\n","").replace("\t","").replace("?","").replace(".","").split(" ")[1:] 
    if((i+1)%3 == 0):
        # get rid of number from the last word
        last_word = tokenized_sent[-1]
        tokenized_sent[-1] = "".join([char for char in last_word if not char.isnumeric()])
    tokens.append(tokenized_sent)

In [9]:
# create a vocabulary from the data
vocab = set()
for sentence in tokens:
    for word in sentence:
        vocab.add(word)
vocab = list(vocab)

# create a dictionary of vocab word indices
word_index = {}
for i, word in enumerate(vocab):
    word_index[word] = i

In [10]:
# function for converting a list of words into a list of word indices
def words_to_indices(words):
    indices = [word_index[word] for word in words]
    return indices

def softmax(x):
    temp = np.exp(x)
    s = np.sum(temp, axis = 1, keepdims = True)
    return  temp / s  


**`Recurrent Neural Nets`**

We will now design a neural network which can be trained to predict the next word in a sentence. To do this, it will be given an input list of words. The forward propagation will involve a sequence of hidden layers (aka `recurrent layers`) corresponding to the given sequence of words. Then we create an `order-dependent sentence embedding` using the method outlined previously, `multiplying each word embedding vector (i.e. hidden layer W0 matrix row corresponding to that word) in the sequence with the recurrance matrix before adding it to the next word vector`, initiating with the "empty sentence" vector (which is just a blank word, i.e. vector of zeros). The resulting sentence embedding forward propagated by each hidden layer can then be multiplied by the output layer weights matrix W1 and operated on by the softmax function to obtain a prediction. So we have a prediction for the next word for every partial sequence of our full sequence of words.

Note that this nueral net design architectire supports variable length input sentences via `variable number of hidden/recurrance layers`. 

In [22]:
class recurrent_net(object):

    def __init__(self, hidden_neurons, vocab) -> None:
       
        input_neurons = output_neurons = len(vocab)

        np.random.seed(1)

        # initialize the hidden layer weights (random number between -0.05 and 0.05)
        self.W0 = 0.1 * (np.random.rand(input_neurons, hidden_neurons) - 0.5)

        # initialize output layer weights  (random number between -0.05 and 0.05) 
        self.W1 = 0.1 *(np.random.rand(hidden_neurons, output_neurons) - 0.5)

        # initialize recurrance matrix to identity matrix
        self.rec = np.eye(hidden_neurons)

        # onhot encoded vocabulary
        self.onehots = np.eye(len(vocab))

        # sentence embedding for an empty sentence
        self.empty_sent = np.zeros(shape=(1,hidden_neurons))
                
        self.vocab = vocab

    # this is the forward propagation poart of our recurrent neural network
    # the input sentence needs to be a list of word indices
    def forward(self, sent):

        # create a list for the variable number of hidden/recurrance layers to support variable length sentences
        layers = []
        layer = {}
        # initialize first hidden layer input as the empty sentence vector
        layer['hidden'] = self.empty_sent
        layers.append(layer)

        loss = 0

        for ix, next_word_index in enumerate(sent):

            layer = {}

            # compute prediction for the next word after this partial sequenc up to the target index
            layer['prediction'] = softmax(np.dot(layers[-1]['hidden'], self.W1))

            # compute the prediction error
            layer['target_index'] =  next_word_index
            layer['error'] = layer['prediction'] - self.onehots[layer['target_index']:layer['target_index']+1]
            
            loss += -np.log(layer['prediction'][0,next_word_index]) 

            # mutliply hidden layer input with recurrance matrix and add the next word vector to generate the input for the next hidden layer
            # (for output layer, we don't need to do this)
            if(ix < (len(sent)-1)):
                layer['hidden'] = np.dot(layers[-1]['hidden'], self.rec) + self.W0[layer['target_index']]

            layers.append(layer)

        return layers, loss  

    # backpropagation part
    def backward(self, layers):

        # starting at the output layer, backpropagate the gradients all the way up to the first hidden layer
        for ix in reversed(range(len(layers))): 

            layer = layers[ix]
            
            if(ix == 0):
                next_layer = layers[ix+1] 

                # dE_dL
                layer['dE_dL'] = next_layer['dE_dL']
                #layer['dE_dL'] = np.dot(next_layer['dE_dL'], rec.T) 

            else:

                # dE_dP
                layer['dE_dP'] = 2 * layer['error']    

                prev_layer = layers[ix-1] 

                # dE_dW1
                layer['dE_dW1'] = np.dot(prev_layer['hidden'].T, layer['dE_dP']) 


                # if this is the output layer, we don't have any gradients backpropagated from any layers in front
                if(ix == (len(layers)-1)):

                    # dE_dL
                    layer['dE_dL'] = np.dot(layer['dE_dP'] , self.W1.T)

                else:

                    next_layer = layers[ix+1] 
                
                    # dE_dW0
                    layer['dE_dW0'] = next_layer['dE_dL']

                    # dE_dR
                    layer['dE_dR'] = np.dot(prev_layer['hidden'].T, next_layer['dE_dL']) 

                    # dE_dL        
                    layer['dE_dL'] = np.dot(next_layer['dE_dL'], self.rec.T) + np.dot(layer['dE_dP'], self.W1.T)   


    # optimization of weights matrices and recurrent matrix
    def optimize(self, layers, sent, alpha):

        # update the empty sentence embedding vector
        self.empty_sent -= (alpha/float(len(sent))) *  layers[0]['dE_dL']

        # apply gradient descent update using the gradients accumulated at each layer
        for ix in range(1,len(layers)): 
            layer = layers[ix]

            self.W1 -= (alpha/float(len(sent))) * layer['dE_dW1'] 
            
            if(ix < (len(layers)-1)):
                # only need to update the weights associated with the target word for that hidden layer
                self.W0[layer['target_index']:layer['target_index']+1] -= (alpha/float(len(sent))) * layer['dE_dW0'] 
                
                self.rec -= (alpha/float(len(sent))) * layer['dE_dR']

                
    def train(self, niters, alpha):

        print("Recurrent neural net training in progress...")

        for iter in range(niters):
            for i, sentence in enumerate(tokens):
                # convert sentence to list of word indices
                indices = words_to_indices(sentence)
                
                # forward propagation
                layers, loss = self.forward(indices)

                # backward propagation
                self.backward(layers)

                # weights optimization
                self.optimize(layers, sentence, alpha)
            
            if(i%500):
                print(f"Iteration# {iter}, Perplexity: {np.exp(loss/len(sentence))}")

        print("Training completed!")        

In [23]:
# initialize a recurrent neural net and trin it
net = recurrent_net(10 , vocab)

net.train(30, 0.001)

Recurrent neural net training in progress...
Iteration# 0, Perplexity: 18.89800661941897
Iteration# 1, Perplexity: 18.779616230757828
Iteration# 2, Perplexity: 18.53216771144119
Iteration# 3, Perplexity: 17.907778507030354
Iteration# 4, Perplexity: 15.84953431105997
Iteration# 5, Perplexity: 13.5382246152571
Iteration# 6, Perplexity: 12.52367836156489
Iteration# 7, Perplexity: 11.24702955848074
Iteration# 8, Perplexity: 9.451328355046604
Iteration# 9, Perplexity: 7.7912395851853695
Iteration# 10, Perplexity: 6.7263832469441835
Iteration# 11, Perplexity: 6.034671943637832
Iteration# 12, Perplexity: 5.594674214241508
Iteration# 13, Perplexity: 5.321957950908765
Iteration# 14, Perplexity: 5.165850353124703
Iteration# 15, Perplexity: 5.072991593660014
Iteration# 16, Perplexity: 4.992249614087307
Iteration# 17, Perplexity: 4.9052028811881145
Iteration# 18, Perplexity: 4.811550864033658
Iteration# 19, Perplexity: 4.713096172618948
Iteration# 20, Perplexity: 4.6105243087232335
Iteration# 21, 